![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir si el precio del automóvil es alto o no. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Importación de librerías
%matplotlib inline
import pandas as pd

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

# Visualización dataset
data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
7,2014,6480,0,0,0,1,0,0,0,1
11,2014,39972,0,0,0,0,1,0,0,0
167,2016,18989,0,0,0,0,0,1,0,1
225,2014,51330,0,0,0,1,0,0,0,0
270,2007,116065,0,1,0,0,0,0,0,0


In [7]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [8]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el acurracy del modelo en el set de test.

In [ ]:
# Celda 1


### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de clasificación y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 2


### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de clasificación y el parámetro `max_features` igual a `log(n_features)`. Presenten el acurracy del modelo en el set de test y comenten sus resultados.

In [ ]:
# Celda 3


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para clasificación y presenten el acurracy del modelo en el set de test y comenten sus resultados.

In [ ]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para clasificación. Presenten el acurracy del modelo en el set de test, comenten sus resultados y análicen cómo cada parámetro afecta el modelo.

In [9]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de clasificación con la librería sklearn, presenten el acurracy del modelo en el set de test y comenten sus resultados.

In [26]:
# Celda 6
from xgboost import XGBClassifier
from sklearn import metrics
clf = XGBClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = metrics.accuracy_score(y_pred, y_test.values)

print('\nAccuracy = {0}'.format(accuracy))

print('\nEste modelo XGBoost, con los parametros default, obtiene un accuracy muy importante {:.4f}.'.format(accuracy))

print('\nParametros utilizados:',clf.get_xgb_params())

[07:57:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Accuracy = 0.8856812933025404

Este modelo XGBoost, con los parametros default, obtiene un accuracy muy importante 0.8857.

Parametros utilizados: {'objective': 'binary:logistic', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'gpu_id': -1, 'interaction_constraints': '', 'learning_rate': 0.300000012, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'monotone_constraints': '()', 'n_jobs': 4, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None}


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para clasificación. Presenten el acurracy del modelo en el set de test, comenten sus resultados y análicen cómo cada parámetro afecta el modelo.

In [123]:
# Celda 7
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold

parameters = {           
              'learning_rate': [0.01, 0.02,0.03,0.05,0.10,0.20,0.30],
              'gamma': [i/10.0 for i in range(1,6)],
              'colsample_bytree': [i/10.0 for i in range(5,11)]
            }
clf = XGBClassifier()
skf = StratifiedKFold(n_splits=5)
n_iter_search = 10
search=RandomizedSearchCV(clf,param_distributions=parameters,n_iter=n_iter_search,
                          scoring='accuracy',n_jobs=-1,cv=skf)
search.fit(X_train, y_train)

best_model = search.best_estimator_

y_pred = best_model.predict(X_test)

cal_accu = metrics.accuracy_score(y_pred, y_test.values)

column_results = [
    f"param_{name}" for name in search.param_distributions.keys()]
column_results += [
    "mean_test_score", "std_test_score", "rank_test_score"]
cv_results = pd.DataFrame(search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False)

def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name

cv_results = cv_results.rename(shorten_param, axis=1)

print('\nEste modelo fue calibrado y los mejores parametros idenficados para colsample_bytree, gamma y learning_rate fueron:')
print('colsample_bytree calibrado:',best_model.get_xgb_params()['colsample_bytree'])
print('gamma calibrado:',best_model.get_xgb_params()['gamma'])
print('learning_rate calibrado:',best_model.get_xgb_params()['learning_rate'])

print('\nAl predecir el set de Test con este modelo calibrado se obtiene un accuracy de {:.4f}.  Mejor que el modelo sin calibracion.'.format(cal_accu))

print('\nEn los multiples escenarios se observa que un learning_rate pequeño genera un mejor score. Mientras que para gamma y col_sample_bytree parecen moverse en valores superiores de la grilla definida según: {0}'.format(search.param_distributions))
print('\nEn la siguiente tabla podemos observar como las diferentes configuraciones de parametros afectan el score para las {0} iteraciones usadas en la calibracion:'.format(n_iter_search))
cv_results


In [117]:
search.param_distributions

{'learning_rate': [0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3],
 'gamma': [0.1, 0.2, 0.3, 0.4],
 'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}

In [62]:
def plot_search_results(grid):
    import matplotlib.pyplot as plt
    """
    Params: 
        grid: A trained GridSearchCV object.
    """
    ## Results from grid search
    results = grid.cv_results_
    means_test = results['mean_test_score']
    stds_test = results['std_test_score']

    ## Getting indexes of values per hyper-parameter
    masks=[]
    masks_names= list(grid.best_params_.keys())
    for p_k, p_v in grid.best_params_.items():
        masks.append(list(results['param_'+p_k].data==p_v))

    params=grid.param_distributions

    ## Ploting results
    fig, ax = plt.subplots(1,len(params),sharex='none', sharey='all',figsize=(20,5))
    fig.suptitle('Score per parameter')
    fig.text(0.04, 0.5, 'MEAN SCORE', va='center', rotation='vertical')
    pram_preformace_in_best = {}
    for i, p in enumerate(masks_names):
        m = np.stack(masks[:i] + masks[i+1:])
        pram_preformace_in_best
        best_parms_mask = m.all(axis=0)
        best_index = np.where(best_parms_mask)[0]
        x = np.array(params[p])
        y_1 = np.array(means_test[best_index])
        e_1 = np.array(stds_test[best_index])
        ax[i].errorbar(x, y_1, e_1, linestyle='--', marker='o', label='test')
        ax[i].set_xlabel(p.upper())

    plt.legend()
    plt.show()

In [103]:
r = search.cv_results_
#cv_results['param_learning_rate']
#search.param_distributions

import plotly.express as px

fig = px.parallel_coordinates(
    cv_results,
    color="mean_test_score",
    dimensions=["mean_test_score", "param_learning_rate"],
    color_continuous_scale=px.colors.diverging.Tealrose,
)
fig.show()


In [59]:
#plot_search_results(search)
results = search.cv_results_

for p_k, p_v in search.best_params_.items():
    print(p_k, p_v)

learning_rate 0.02
gamma 0.3
colsample_bytree 0.6


In [42]:
import numpy as np

kk = XGBClassifier(colsample_bytree=0.6,
              gamma=0.3, 
              learning_rate=0.02)

kk.fit(X_train, y_train)
y_pred = kk.predict(X_test)
accuracy = metrics.accuracy_score(y_pred, y_test.values)

print('\nEste modelo fue calibrado y los mejores parametros idenficados para colsample_bytree, gamma y learning_rate fueron:')
print('colsample_bytree calibrado:',kk.get_xgb_params()['colsample_bytree'])
print('gamma calibrado:',kk.get_xgb_params()['gamma'])
print('learning_rate calibrado:',kk.get_xgb_params()['learning_rate'])

print('\nAl predecir el set de Test con este modelo calibrado se obtiene un accuracy de {:.4f}.'.format(accuracy))



C:\Users\geiber.TREBOL\miniconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:27:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Este modelo fue calibrado y los mejores parametros idenficados para colsample_bytree, gamma y learning_rate fueron:
colsample_bytree calibrado: 0.6
gamma calibrado: 0.3
learning_rate calibrado: 0.02

Al predecir el set de Test con este modelo calibrado se obtiene un accuracy de 0.8874.


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 8
